# S2_Get_Geo_locator_from_address.ipynb
## Descriptions
* Address as string
* Convert to geo locator using osm 

In [1]:
import sqlite3 as lite
def select_addr(conn):
    """
    """
    cur = conn.cursor()
    cur.execute("select ci_text from corp_info where ci_name in ('REGISTERED OFFICE ADDRESS','DIRECTORS')") 
    rows = cur.fetchall()
    return([r[0] for r in rows])

ddb = "./data/corporate/corp.db"                    

con = lite.connect(ddb)
l_addr = list(set(select_addr(con)))
l_addr = l_addr[1:]
con.close()

In [2]:

l_addr[:100]

['8553 DANIEL-DONY MONTRÉAL QC H1E 6W1 CANADA',
 '16 RUE DUMONT GATINEAU QC J8V 1N5 CANADA',
 '11 HUTTONVILLE DR. BRAMPTON ON L6V 3N2 CANADA',
 '3400 LAKESHORE BLVD WEST APT 216 ETOBICOKE ON M8W 4Z9 CANADA',
 '341 KENASTON TOWN OF MONT ROYAL QC H3R 1N3 CANADA',
 '2389 PROUDFOOT TRAIL OAKVILLE ON L6M 4Y2 CANADA',
 '215 RUE SAINT-JACQUES BUR. 1200 MONTREAL QC H2Y 1M6 CANADA',
 '104 COLONIAL GATINEAU QC J9H 0A6 CANADA',
 '42 SURREY DRIVE MONT ROYAL QC H3P 1B1 CANADA',
 '11, GROSVENOR ROAD DORCHESTER, DORSET DT12BB UNITED KINGDOM',
 '1877 FEATHERSTONE DRIVE MISSISSAUGA ON L5L 2T2 CANADA',
 '17 NASIR STREET MAPLE ON L6A 3A1 CANADA',
 'CARE OF: ALEC YU 23 HILDENBORO SQ. TORONTO ON M1W 1Y2 CANADA',
 '1137 HOWARD AVE WINDSOR ON N9A 1S6 CANADA',
 '5724 MERRIMAC COTE ST-LUC QC CANADA',
 'CARE OF: COMMERCE CRT W. SUITE 5300 PO BOX 85 TORONTO ON M5L 2B9 CANADA',
 '2525 AVENUE HAVRE-DES-ILES APP 803 LAVAL QC H7W 4C6 CANADA',
 '67 PURPLE DUSK TRAIL SCARBOROUGH ON M1E 4C6 CANADA',
 '935 MARINE DRIVE 

In [20]:
import geocoder

i = 0
for addr in l_addr:    
    g = geocoder.osm(addr)
    with open("./data/corporate/address.csv","a") as file_w:
        if g.json is None:
            file_w.write(str(i) + "," + ' "' + addr + '",' + '"' + "" + '"' + "\n")
        else:
            file_w.write(str(i) + "," + ' "' + addr + '",' + '"' + str(g.json) + '"' + "\n")
    i+=1
